In [35]:
import os
import pandas as pd

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR

check_and_make_directories([TRAINED_MODEL_DIR])

In [36]:
train = pd.read_csv('./train_data/train_data.csv')
test = pd.read_csv('./test_data/test_data.csv')

In [37]:
train

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,AAPL,6.424606,6.439316,6.375674,6.407195,493729600.0,0.0,0.000000,6.445867,6.414452,100.000000,66.666667,100.000000,6.424606,6.424606,20.040001,0.000000
1,2010-01-04,AMZN,6.695000,6.830500,6.657000,6.812500,151998000.0,0.0,0.000000,6.445867,6.414452,100.000000,66.666667,100.000000,6.695000,6.695000,20.040001,0.000000
2,2010-01-04,GOOGL,15.576999,15.645595,15.514617,15.581969,78169752.0,0.0,0.000000,6.445867,6.414452,100.000000,66.666667,100.000000,15.576999,15.576999,20.040001,0.000000
3,2010-01-04,MSFT,23.173340,23.285650,22.903795,22.926257,38409100.0,0.0,0.000000,6.445867,6.414452,100.000000,66.666667,100.000000,23.173340,23.173340,20.040001,0.000000
4,2010-01-05,AAPL,6.435713,6.472038,6.401790,6.442318,601904800.0,1.0,0.000249,6.445867,6.414452,100.000000,66.666667,100.000000,6.430160,6.430160,19.350000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13083,2022-12-29,MSFT,235.822784,236.713202,230.578146,230.578146,19770700.0,3.0,-1.317345,253.969008,227.311662,48.757455,-79.801944,5.313316,240.140495,234.343710,21.440001,1.905451
13084,2022-12-30,AAPL,128.123062,128.142788,125.657837,126.624212,77034200.0,4.0,-4.343328,148.692628,122.196968,41.597135,-121.278276,29.024002,138.980270,140.738595,21.670000,0.309391
13085,2022-12-30,AMZN,84.000000,84.050003,82.470001,83.120003,62401200.0,4.0,-3.347538,94.472016,80.645983,39.422059,-115.829424,25.711482,89.725999,97.755667,21.670000,0.309391
13086,2022-12-30,GOOGL,87.625641,87.695161,85.977008,86.384203,23986300.0,4.0,-2.270884,99.523770,82.980479,42.973902,-104.403756,16.912089,93.197540,94.108424,21.670000,0.309391


In [38]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 41


In [39]:
train = train.reset_index(drop=True)

In [40]:
print(train.index)

RangeIndex(start=0, stop=13088, step=1)


In [41]:
print(train.loc[0, "macd"])


0.0


In [42]:
train.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,AAPL,6.424606,6.439316,6.375674,6.407195,493729600.0,0.0,0.000000,6.445867,6.414452,100.0,66.666667,100.0,6.424606,6.424606,20.040001,0.0
1,2010-01-04,AMZN,6.695000,6.830500,6.657000,6.812500,151998000.0,0.0,0.000000,6.445867,6.414452,100.0,66.666667,100.0,6.695000,6.695000,20.040001,0.0
2,2010-01-04,GOOGL,15.576999,15.645595,15.514617,15.581969,78169752.0,0.0,0.000000,6.445867,6.414452,100.0,66.666667,100.0,15.576999,15.576999,20.040001,0.0
3,2010-01-04,MSFT,23.173340,23.285650,22.903795,22.926257,38409100.0,0.0,0.000000,6.445867,6.414452,100.0,66.666667,100.0,23.173340,23.173340,20.040001,0.0
4,2010-01-05,AAPL,6.435713,6.472038,6.401790,6.442318,601904800.0,1.0,0.000249,6.445867,6.414452,100.0,66.666667,100.0,6.430160,6.430160,19.350000,0.0


In [43]:
train["day"] = train["day"].astype(int)
train = train.reset_index(drop=True)

In [45]:
buy_cost_list = sell_cost_list = [0.001]*stock_dimension
num_stock_shares = [0]*stock_dimension

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": INDICATORS, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

AttributeError: 'numpy.float64' object has no attribute 'values'

In [49]:
train.dtypes

date             object
tic              object
close           float64
high            float64
low             float64
open            float64
volume          float64
day               int32
macd            float64
boll_ub         float64
boll_lb         float64
rsi_30          float64
cci_30          float64
dx_30           float64
close_30_sma    float64
close_60_sma    float64
vix             float64
turbulence      float64
dtype: object